# Thesis Code Part 4: BERT

In [ ]:
!pip install transformers
!pip install evaluate

In [ ]:
train = pd.read_csv('data/train_dataset.csv')
validation = pd.read_csv('data/val_dataset.csv')
test = pd.read_csv('data/test_dataset.csv')

In [ ]:
train_texts, train_labels = train['clean_text'].tolist(), train['label'].tolist()
val_texts, val_labels = validation['clean_text'].tolist(), validation['label'].tolist()
test_texts, test_labels = test['clean_text'].tolist(), test['label'].tolist()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("base-uncased")
model = BertForSequenceClassification.from_pretrained("base-uncased",num_labels=2)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [ ]:
import torch

class ReportsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReportsDataset(train_encodings, train_labels)
val_dataset = ReportsDataset(val_encodings, val_labels)
test_dataset = ReportsDataset(test_encodings, test_labels)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

In [ ]:
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
import evaluate
import numpy as np
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)